In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

/Users/c4103145/Documents/hackathon/venv/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


,carpark,total_items,idle_hours,total_on_till_hours,total_transaction_count
store_id,,,,,
37,1.0,78881.663462,85.822829,236.759319,9175.990385
38,1.0,64774.625000,76.529260,185.660409,7478.067308
54,1.0,48444.278846,55.794837,146.601306,6685.336538
55,1.0,128745.615385,90.282476,313.043112,9567.961538
57,1.0,116904.836538,90.833235,281.287166,11069.105769
...,...,...,...,...,...
9894,1.0,93538.413462,75.896985,234.486998,7747.663462
9920,0.0,93934.038462,113.542545,272.654778,7558.442308
9946,1.0,36550.067308,48.039415,116.290887,5989.182692


In [3]:
data = pd.read_csv('../data/raw/train_test.csv')
data['week_start'] =  pd.to_datetime(data['week_start'])

lookup = data.drop(columns = ['week_start','total_items','idle_hours','total_on_till_hours','total_transaction_count'])
data = data.drop(columns = ['idle_hours','total_transaction_count','total_items'])

lookup_unique = lookup.drop_duplicates()

lookup_unique_OHE = pd.get_dummies(lookup_unique)
lookup_unique_OHE = lookup_unique_OHE.set_index('store_id')
data_OHE = pd.get_dummies(data)

In [66]:
data_OHE['day'] = data_OHE['week_start'].dt.day
data_OHE['month'] = data_OHE['week_start'].dt.month
data_OHE['year'] = data_OHE['week_start'].dt.year

data_OHE= data_OHE.drop(columns=['week_start'])

X = data_OHE.drop(columns=['total_on_till_hours'])
y = np.ravel(data_OHE[['total_on_till_hours']])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [67]:
model = XGBRegressor(random_state = 42, n_jobs = -1)

In [68]:
model.fit(X_train,y_train)

/Users/c4103145/Documents/hackathon/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=-1,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [69]:
predictions = model.predict(X_test)

/Users/c4103145/Documents/hackathon/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [70]:
mean_absolute_error(y_true=y_test, y_pred=predictions)

17.627557679177656

In [71]:
testing_original = pd.read_csv('../data/raw/result.csv')
testing_original['week_start'] =  pd.to_datetime(testing_original['week_start'])

testing_df = testing_original.copy()

testing_df['day'] = testing_df['week_start'].dt.day
testing_df['month'] = testing_df['week_start'].dt.month
testing_df['year'] = testing_df['week_start'].dt.year
testing_df = testing_df.set_index('store_id')

In [72]:
testing_df = testing_df.join(lookup_unique_OHE)

In [73]:
testing_df_for_predict = testing_df.drop(columns=['week_start','predicted_total_on_till_hours'])

In [74]:
testing_df_for_predict = testing_df_for_predict.reset_index()

In [76]:
testing_predictions = model.predict(testing_df_for_predict)

/Users/c4103145/Documents/hackathon/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [81]:
testing_original['predicted_total_on_till_hours'] = testing_predictions

In [83]:
output_df = testing_original[['week_start','predicted_total_on_till_hours']]

In [86]:
output_df = output_df.reset_index()

In [87]:
output_df.to_csv('~/Desktop/output.csv')